In [48]:
pip install qiskit 

In [50]:
import numpy as np
import qiskit 
from qiskit_aer import AerSimulator
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit.library import ZZFeatureMap
from qiskit.primitives import Sampler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd


In [51]:
def load_twitter_data(file_path):
    df = pd.read_csv(file_path)
    df = df.dropna(subset=['text'])
    X = df['text'].values
    y = df['sentiment'].values
    return X, y

In [52]:
def preprocess_data(X, y, max_features=100):
    y = (y == 'positive').astype(int)
    
    vectorizer = CountVectorizer(max_features=max_features)
    X = vectorizer.fit_transform(X)
    
    if X.shape[1] < max_features:
        padding = np.zeros((X.shape[0], max_features - X.shape[1]))
        X = np.hstack([X.toarray(), padding])
    else:
        X = X.toarray()
    
    # Normalize each sample
    X = X / np.linalg.norm(X, axis=1)[:, np.newaxis]
    
    return X, y, vectorizer

In [53]:
def quantum_conv_layer(qc, qubits, params):
    for i in range(len(qubits) - 1):
        qc.rzz(params[i], qubits[i], qubits[i+1])
    return qc

In [54]:
def quantum_pool_layer(qc, qubits):
    for i in range(0, len(qubits) - 1, 2):
        qc.cx(qubits[i], qubits[i+1])
        qc.measure(qubits[i+1], i//2)
    return qc

In [56]:
def create_quantum_circuit(num_qubits, params):
    qr = QuantumRegister(num_qubits)
    cr = ClassicalRegister(num_qubits // 2)
    qc = QuantumCircuit(qr, cr)
    
    feature_map = ZZFeatureMap(num_qubits)
    qc.append(feature_map, qr)
    
    qc = quantum_conv_layer(qc, qr, params[:num_qubits-1])
    qc = quantum_pool_layer(qc, qr)
    
    return qc

In [57]:
class QuantumCNN:
    def __init__(self, num_qubits, shots=1000):
        self.num_qubits = num_qubits
        self.shots = shots
        self.params = np.random.randn(num_qubits - 1)
        self.sampler = Sampler()
    
    def get_expectation(self, x):
        qc = create_quantum_circuit(self.num_qubits, self.params)
        qc.initialize(x, qc.qubits)
        
        job = self.sampler.run(qc, shots=self.shots)
        result = job.result().quasi_dists[0]
        
        expectation = sum([int(bitstring, 2) * count for bitstring, count in result.items()]) / self.shots
        return expectation
    
    def predict(self, X):
        return np.array([self.get_expectation(x) for x in X])



In [59]:
def train_quantum_cnn(X_train, y_train, num_qubits, epochs=50, learning_rate=0.01):
    model = QuantumCNN(num_qubits)
    
    for epoch in range(epochs):
        y_pred = model.predict(X_train)
        loss = np.mean((y_pred - y_train) ** 2)
        
        grad = np.zeros_like(model.params)
        epsilon = 1e-6
        for i in range(len(model.params)):
            model.params[i] += epsilon
            y_pred_plus = model.predict(X_train)
            model.params[i] -= 2 * epsilon
            y_pred_minus = model.predict(X_train)
            model.params[i] += epsilon
            
            grad[i] = np.mean((y_pred_plus - y_pred_minus) * (y_train - y_pred)) / (2 * epsilon)
        
        model.params -= learning_rate * grad
        
        if epoch % 10 == 0:
            print(f"Epoch {epoch}, Loss: {loss:.4f}")
    
    return model

In [60]:
if __name__ == "__main__":
    X, y = load_twitter_data("twittersentiment.csv")
    X, y, vectorizer = preprocess_data(X, y, max_features=8)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    num_qubits = 8
    model = train_quantum_cnn(X_train, y_train, num_qubits, epochs=100)

    y_pred = model.predict(X_test)
    y_pred_binary = (y_pred > 0.5).astype(int)
    accuracy = accuracy_score(y_test, y_pred_binary)
    print(f"Test Accuracy: {accuracy:.4f}")

C:\Users\aarya\AppData\Local\Temp\ipykernel_11716\4054517441.py:14: RuntimeWarning: invalid value encountered in divide
  X = X / np.linalg.norm(X, axis=1)[:, np.newaxis]
C:\Users\aarya\AppData\Local\Temp\ipykernel_11716\294637842.py:6: DeprecationWarning: The class ``qiskit.primitives.sampler.Sampler`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseSamplerV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Sampler` class is `StatevectorSampler`.
  self.sampler = Sampler()


QiskitError: 'StatePreparation parameter vector has 8 elements, therefore expects 3 qubits. However, 8 were provided.'